# DSCI D 699 Independent Study in Data Science Fall 2018

## Advisor: Professor David Crandall

Author: Aniruddha M Godbole


In this notebook the proxy personality trait representative vectors are created. 
Please see 6B.py along with this notebook. In 7.py the 160 scores (150 scores for the 150 topics and 10 scores for the proxy personality traits were computed. This was computationally intensive as for each complaint for each type of loan each word was considered for computing a similarity with the proxy personality trait vector) and this could not be run in this notebook.

In [1]:
import os
import csv
import pandas as pd
import matplotlib.pyplot as plt
import gensim
import numpy as np

from gensim.models import CoherenceModel, LdaModel, LsiModel, HdpModel
from gensim.models.wrappers import LdaMallet
from gensim.corpora import Dictionary
import pyLDAvis.gensim

import os, re, operator, warnings
warnings.filterwarnings('ignore')  
%matplotlib inline

C:\Users\agodbole\Anaconda3\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
import pickle

In [3]:
payday_hdpmodel=HdpModel.load('./data/payday_hdpmodel')
student_hdpmodel=HdpModel.load('./data/student_hdpmodel')
mortgage_hdpmodel=HdpModel.load('./data/mortgage_hdpmodel')

In [4]:
#https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_pickle.html
dfmortproddebtcoll_common_cos=pd.read_pickle('./data/dfmortproddebtcoll_common_cos.pickle')
dfpayproddebtcoll_common_cos=pd.read_pickle('./data/dfpayproddebtcoll_common_cos.pickle')
dfstudproddebtcoll_common_cos=pd.read_pickle('./data/dfstudproddebtcoll_common_cos.pickle')

In [5]:
#https://stackoverflow.com/questions/4530611/saving-and-loading-objects-and-using-pickle
payday_filehandler = open('./data/payday_dictionary.pickle','rb')
payday_dictionary = pickle.load(payday_filehandler)
payday_filehandler.close()

student_filehandler = open('./data/student_dictionary.pickle','rb')
student_dictionary = pickle.load(student_filehandler)
student_filehandler.close()

mortgage_filehandler = open('./data/mortgage_dictionary.pickle','rb')
mortgage_dictionary = pickle.load(mortgage_filehandler)
mortgage_filehandler.close()

In [6]:
import nltk

In [7]:
from nltk.corpus import sentiwordnet as swn 

In [8]:
#https://github.com/bhargavvader/personal/blob/master/notebooks/text_analysis_tutorial/topic_modelling.ipynb
#https://www.youtube.com/watch?v=ZkAFJwi-G98
import spacy
nlp = spacy.load('en_core_web_sm')

In [9]:
dfmortproddebtcoll_common_cos_ratings=pd.read_pickle('./data/dfmortproddebtcoll_common_cos_ratings.pickle')
dfstudproddebtcoll_common_cos_ratings=pd.read_pickle('./data/dfstudproddebtcoll_common_cos_ratings.pickle')
dfpayproddebtcoll_common_cos_ratings=pd.read_pickle('./data/dfpayproddebtcoll_common_cos_ratings.pickle')

In [10]:
model_tweet_to_complaint=gensim.models.KeyedVectors.load_word2vec_format('./data/tweet_complaint_word_vec/tweet_to_complaint.vec')

In [12]:
model_tweet_to_complaint['lol']

array([ 6.60802e-02, -5.02282e-01, -4.18075e-01,  7.51906e-02,
       -8.85061e-02, -4.86695e-03,  6.41264e-02,  5.00847e-02,
        8.42760e-02, -9.10691e-02, -1.40521e-01, -3.64634e-02,
       -8.46322e-03, -1.37677e-02, -5.99100e-02,  2.33199e-02,
        1.23624e-01, -3.82488e-03, -1.28981e-02, -1.34688e-02,
        6.14677e-02,  1.06221e-02, -1.05918e-02,  3.01735e-02,
       -2.33288e-02,  3.04884e-02, -6.44557e-03,  3.13371e-03,
       -9.64211e-03,  6.92522e-02, -3.39395e-02, -1.56403e-02,
       -4.54064e-03, -3.23487e-02, -9.90031e-03, -8.43808e-03,
       -7.46452e-02,  7.05918e-02,  8.06843e-02, -2.56275e-02,
        1.73340e-02, -3.82487e-02,  1.40251e-02, -4.03672e-02,
       -2.91025e-02,  3.55041e-02,  1.91801e-02,  3.18172e-02,
        6.21602e-02,  1.42704e-02,  5.02564e-02,  1.01882e-02,
       -5.66533e-03,  9.83909e-02,  8.89165e-04,  4.04426e-04,
       -1.50456e-02,  2.65248e-02, -6.07973e-02,  7.63207e-02,
        3.69114e-03,  8.25633e-03,  2.68929e-02, -1.300

In [13]:
model_tweet_to_complaint.most_similar('lol')

[('lmao', 0.9058093428611755),
 ('lmaoyeah', 0.8773184418678284),
 ('lolâ«lmao', 0.8771474361419678),
 ('lolâ»', 0.8760850429534912),
 ('lmaolol', 0.8759668469429016),
 ('smhlol', 0.8753378391265869),
 ('â«lol', 0.8718224763870239),
 ('lmaooh', 0.8688886165618896),
 ('lmaoyea', 0.8667067885398865),
 ('cuzlol', 0.8635356426239014)]

In [14]:
model_tweet_to_complaint.most_similar('love')

[('lovex', 0.7380396127700806),
 ('love@', 0.7372550368309021),
 ('loveif', 0.7371676564216614),
 ('loveu*', 0.7337770462036133),
 ('lovee*', 0.7326759099960327),
 ('lovea', 0.6921778917312622),
 ('loveor', 0.6904361844062805),
 ('loveso', 0.6875289678573608),
 ('loveme', 0.6853914856910706),
 ('lovesit', 0.6829686164855957)]

In [15]:
model_tweet_to_complaint.most_similar('cant')

[("can't", 0.8614391088485718),
 ('cnt', 0.819844663143158),
 ("ican't", 0.7981957197189331),
 ("can'ti", 0.7566649913787842),
 ("mecan't", 0.7476279735565186),
 ('dont', 0.738701581954956),
 ('icouldnt', 0.7344928979873657),
 ('couldnt', 0.7308478355407715),
 ('wont', 0.7285248041152954),
 ("caaan't", 0.7248843908309937)]

In [16]:
model_tweet_to_complaint.similarity('cheat','fraud')

0.20456266

In [17]:
model_tweet_to_complaint.similarity('threat','cheat')

0.24117683

In [18]:
model_complaint_to_tweet=gensim.models.KeyedVectors.load_word2vec_format('./data/tweet_complaint_word_vec/complaint_to_tweet.vec')

In [19]:
model_complaint_to_tweet['lol']

array([-0.064749  , -0.105284  , -0.130971  ,  0.0322671 ,  0.0121459 ,
        0.00480626,  0.0769705 ,  0.0429713 , -0.13548   ,  0.130559  ,
       -0.0593473 ,  0.0456342 , -0.0113554 ,  0.0140262 , -0.111757  ,
        0.0609615 , -0.0237023 ,  0.0444893 , -0.00826049, -0.168311  ,
       -0.0256443 ,  0.0666228 ,  0.0044491 , -0.0509502 , -0.011836  ,
        0.0294192 ,  0.0647614 ,  0.144449  , -0.0147986 ,  0.04108   ,
        0.030569  , -0.0690208 , -0.10902   , -0.0831267 , -0.108953  ,
        0.0221024 , -0.0658965 , -0.00325213,  0.0290174 ,  0.0785924 ,
       -0.0185333 , -0.00067272,  0.00058521,  0.073093  , -0.066947  ,
        0.0378274 , -0.0634177 ,  0.0436411 ,  0.00904366,  0.00359186,
        0.00647265,  0.105444  , -0.0203722 ,  0.0385912 , -0.0527248 ,
       -0.00705061,  0.0397114 ,  0.0719766 , -0.0120904 , -0.0555129 ,
        0.0334071 , -0.00101237,  0.0543183 ,  0.000762  , -0.0329606 ,
        0.0408442 , -0.0545797 , -0.0190762 , -0.0141977 , -0.04

In [20]:
model_complaint_to_tweet['cheat']

array([ 0.0957713 ,  0.0530647 , -0.159882  , -0.160965  , -0.059435  ,
       -0.107097  ,  0.055229  ,  0.0398519 , -0.0579238 ,  0.0881777 ,
       -0.157189  ,  0.0777059 ,  0.178847  ,  0.0801706 ,  0.0909239 ,
        0.111304  , -0.00538078, -0.00976121, -0.0866969 , -0.148369  ,
       -0.0649711 ,  0.0543257 ,  0.0589641 , -0.0311044 ,  0.110692  ,
        0.0366802 ,  0.0675168 ,  0.0839088 ,  0.0281989 ,  0.027531  ,
        0.0407005 ,  0.0257235 , -0.0315607 , -0.0067909 , -0.0649112 ,
        0.0572909 ,  0.0850196 ,  0.0392325 , -0.00624096,  0.031353  ,
        0.00650148,  0.0519773 , -0.00810186,  0.0426557 , -0.0294298 ,
        0.0997215 ,  0.0295998 , -0.0698734 , -0.100821  ,  0.0821607 ,
        0.0365859 ,  0.0230415 , -0.0431175 , -0.00942904,  0.0300334 ,
        0.0040638 , -0.00672505,  0.0104887 ,  0.0763612 ,  0.0710589 ,
       -0.0187194 ,  0.0125897 , -0.0253661 , -0.00884776, -0.0323911 ,
        0.0413242 ,  0.046617  , -0.00334809,  0.00785769, -0.04

In [21]:
model_complaint_to_tweet.similarity('cheat','fraud')

0.17133917

In [22]:
model_complaint_to_tweet.similarity('cheat','threat')

0.20505065

In [23]:
model_complaint_to_tweet.similarity('cheat','cheating')

0.7883131

# Agreeableness

In [24]:
model_complaint_to_tweet.similarity('blessed','excited')

0.553423

In [25]:
model_complaint_to_tweet.similarity('hell','excited')

0.334295

In [26]:
model_complaint_to_tweet.similarity('hell','blessed')

0.29559517

In [27]:
model_complaint_to_tweet.similarity('smoke','blessed')

0.26155198

In [28]:
model_complaint_to_tweet.similarity('joy','excited')

0.46964723

In [29]:
model_complaint_to_tweet.similarity('damned','excited')

0.35822487

# Extraversion

In [30]:
model_complaint_to_tweet['party']

array([ 1.00898e-01,  9.17673e-02,  6.55390e-02,  9.35383e-02,
        9.15899e-02, -1.31185e-01,  2.58256e-02, -2.29656e-02,
       -4.58423e-02,  4.28538e-04, -1.39856e-01, -7.60818e-02,
       -1.86832e-02, -5.74736e-02, -5.75533e-02,  9.77586e-03,
        8.94865e-02,  6.34313e-02, -3.76777e-02, -3.62342e-02,
        2.09884e-02,  5.35382e-02,  5.98215e-02,  5.97796e-02,
        7.73621e-03, -4.98013e-02, -9.68099e-02, -1.08866e-02,
       -3.67744e-03, -1.24905e-02, -4.23875e-02,  1.70070e-02,
        1.14289e-02,  1.47006e-02,  3.58879e-02, -4.76452e-02,
        7.38283e-04,  4.02287e-02, -1.07205e-03,  1.44144e-02,
       -1.03667e-01, -2.19165e-02,  7.78688e-03,  2.21430e-02,
        3.59328e-02,  2.19780e-02,  4.95488e-02,  5.17758e-02,
       -1.40781e-02,  3.50147e-03, -1.47350e-02,  1.35560e-01,
       -1.34680e-02, -3.88540e-02, -4.96648e-02,  6.02172e-02,
       -6.15460e-02,  5.98181e-02,  3.65144e-02, -5.63599e-02,
        1.01636e-02, -1.64763e-02,  2.13578e-02, -2.001

In [31]:
model_complaint_to_tweet.similarity('party','internet')

0.0072844555

In [32]:
model_complaint_to_tweet.similarity('party',"didn't")

-0.06241451

In [33]:
model_complaint_to_tweet.similarity('party','gym')

0.04851552

# Neuroticism

In [34]:
model_complaint_to_tweet['success']

array([ 1.13248e-01,  1.25855e-01, -1.42088e-01,  2.99313e-02,
        7.46378e-02, -2.96580e-02, -1.85201e-01,  6.08364e-02,
        2.77415e-02, -5.64812e-02,  1.98837e-01,  1.40317e-04,
       -7.34919e-02, -3.11565e-02,  7.49107e-02, -1.82615e-02,
       -3.14878e-02,  5.52598e-02,  5.87965e-03, -5.87478e-02,
        7.42318e-02, -5.50650e-02,  4.27741e-03, -5.48765e-02,
       -1.11663e-02, -7.36387e-02,  5.50523e-02,  1.15782e-01,
       -1.90508e-02, -2.15302e-02,  5.66660e-02,  1.20001e-02,
        3.16423e-02,  1.19680e-02, -9.48805e-02, -1.05524e-01,
        1.07057e-01, -8.35356e-02,  3.72380e-02,  1.24299e-02,
        4.11712e-02,  7.31445e-03,  2.07862e-02,  2.91073e-02,
       -1.80928e-02, -1.87069e-02,  5.05262e-03, -3.12348e-02,
        7.07704e-03, -9.81910e-02, -1.11045e-02,  2.64483e-02,
       -1.92867e-02,  3.09322e-02,  2.79323e-02, -8.08706e-05,
        2.91418e-02,  2.97784e-02, -2.00475e-02,  2.47776e-02,
        5.74453e-02, -1.58003e-02,  6.83873e-03, -6.315

In [35]:
model_complaint_to_tweet.similarity('success','mountains')

0.09857944

In [36]:
model_complaint_to_tweet.similarity('success','hurts')

0.20054452

In [37]:
model_complaint_to_tweet.similarity('success','pills')

-0.03266134

In [38]:
model_complaint_to_tweet.similarity('success','depressed')

0.11781454

# Conscientiousness

In [39]:
model_complaint_to_tweet['blessed']

array([ 1.32499e-01, -9.42031e-02, -1.91132e-01, -2.81321e-02,
       -4.01701e-03,  5.38021e-02, -7.52385e-02,  1.20690e-01,
       -1.40528e-01, -4.43650e-02,  5.04393e-02,  4.30855e-02,
       -1.17026e-01,  3.17466e-02, -7.59761e-02, -1.69914e-02,
       -1.20089e-01, -2.74659e-02, -1.10497e-01,  1.12021e-01,
       -1.18384e-01, -1.99713e-01, -3.33645e-02, -5.83022e-02,
        8.33817e-02, -1.17802e-01,  7.68662e-02, -5.54642e-02,
       -1.55100e-01,  4.29085e-02,  1.71081e-03, -5.62107e-02,
       -6.55668e-02, -6.51934e-02, -6.83679e-02, -9.82152e-03,
        7.09549e-02,  9.15588e-02, -1.28630e-01, -6.28408e-02,
        1.90895e-02,  4.75561e-02,  1.25755e-02,  3.82315e-02,
       -5.47760e-02, -2.78432e-02, -1.09776e-01, -3.72600e-02,
        4.80977e-02, -4.75368e-02,  1.72857e-02,  6.25387e-03,
        6.60047e-03,  8.84169e-02, -1.23017e-02, -7.97167e-02,
       -3.57474e-02, -5.96629e-02,  1.71333e-02, -5.20325e-02,
        4.30204e-02, -8.56719e-03,  2.33217e-02,  5.362

In [40]:
model_complaint_to_tweet.similarity('blessed','strength')

0.18908711

In [41]:
model_complaint_to_tweet.similarity('blessed','black')

0.07255644

In [42]:
model_complaint_to_tweet.similarity('blessed','bored')

0.31303397

In [43]:
model_complaint_to_tweet.similarity('strength','bored')

0.1453625

In [44]:
model_complaint_to_tweet.similarity('black','bored')

0.25291803

In [45]:
model_tweet_word=gensim.models.KeyedVectors.load_word2vec_format('./data/tweet_complaint_word_vec/tweet_word.vec')

In [46]:
model_complaint_word=gensim.models.KeyedVectors.load_word2vec_format('./data/tweet_complaint_word_vec/complaint_word.vec')

In [47]:
model_tweet_word['fuck']

array([ 1.5250e-01,  1.4763e-01, -2.9234e-01,  1.9766e-01, -1.6851e-01,
        1.6862e-01, -2.5565e-01,  1.3825e-02,  4.1039e-01,  1.5860e-01,
       -7.8938e-03,  3.0830e-01, -2.6350e-01, -2.0329e-01, -2.6728e-01,
       -1.8353e-01,  2.1038e-01, -2.2962e-01,  3.0643e-01, -3.3632e-02,
       -2.6361e-01,  1.5626e-01,  3.2831e-01, -9.5369e-02, -1.3993e-01,
        3.9630e-01,  4.1505e-03,  2.2960e-01, -3.9550e-01,  3.1769e-01,
        1.3611e-01,  1.1278e-01, -1.8999e-01, -2.5272e-01, -3.1995e-01,
        3.5336e-02,  3.3776e-01,  1.3849e-01, -4.1700e-02, -3.2803e-02,
        1.8639e-01,  1.1401e-01, -1.4656e-01,  8.0635e-02, -1.7064e-01,
        3.3951e-01, -7.3463e-01, -2.3746e-01, -3.9117e-01,  3.1081e-01,
       -4.8311e-02, -3.7542e-01, -4.2085e-01,  2.7409e-01, -1.9784e-01,
       -3.4198e-01, -1.1271e-01, -3.3651e-01, -7.6536e-02,  6.6875e-02,
        8.5558e-02,  4.8638e-01,  2.7710e-01, -1.5263e-01, -1.3357e-01,
        5.1600e-02, -4.2345e-02, -4.2404e-01, -1.3374e-02,  6.57

In [48]:
model_tweet_word.most_similar('fuck')

[('fuckshit', 0.8116024136543274),
 ('fucku', 0.8083146214485168),
 ('fuckry', 0.8046603202819824),
 ("fuck'd", 0.801166296005249),
 ('bitchfuck', 0.7975711822509766),
 ('fuckz', 0.7953677177429199),
 ('wtfuck', 0.7906858921051025),
 ('fuckkn', 0.7897753715515137),
 ('ffuck', 0.7866847515106201),
 ('fucx', 0.7776360511779785)]

In [49]:
model_tweet_word.similarity('black','bored')

0.14168434

In [50]:
model_tweet_word['space']

array([ 0.05349  ,  0.1866   ,  0.2563   ,  0.5117   ,  0.25577  ,
        0.36635  ,  0.38866  ,  0.32432  ,  0.062101 ,  0.47236  ,
       -0.06272  , -0.03242  , -0.044681 ,  0.19071  ,  0.22583  ,
        0.20015  , -0.25629  , -0.36745  ,  0.27347  ,  0.34701  ,
        0.010059 , -0.018663 , -0.061183 , -0.15435  ,  0.14057  ,
       -0.1255   ,  0.20964  ,  0.3234   , -0.42316  ,  0.36733  ,
       -0.11213  ,  0.80903  ,  0.0034858,  0.16348  , -0.12741  ,
       -0.26135  , -0.36097  ,  0.2096   ,  0.18408  , -0.14609  ,
       -0.13447  , -0.036193 ,  0.4081   ,  0.5745   ,  0.11586  ,
        0.35596  ,  0.65876  , -0.22208  , -0.11963  , -0.21115  ,
       -0.15273  ,  0.24986  , -0.24451  , -0.0061249, -0.31708  ,
       -0.19006  ,  0.070302 ,  0.34397  ,  0.3945   , -0.23773  ,
        0.22158  , -0.16866  , -0.66926  , -0.23102  ,  0.3096   ,
        0.34411  , -0.35127  , -0.31217  ,  0.0041206,  0.1462   ,
        0.051355 , -0.19603  ,  0.049115 ,  0.35766  ,  0.0415

In [51]:
model_tweet_word['pokemon']

array([-1.1730e-02,  3.5788e-01, -1.4313e-01,  6.1754e-02, -1.3268e-01,
        3.7070e-02,  2.2857e-01, -6.3834e-01, -2.2217e-02,  4.9946e-01,
        8.0802e-02, -3.6610e-01,  3.5605e-01,  1.7760e-01,  3.4181e-01,
       -1.4608e-02, -2.4579e-01, -1.6401e-01, -2.6097e-01, -2.9466e-02,
        1.6684e-01,  3.8753e-01, -3.1689e-01, -3.3979e-01, -3.7430e-01,
       -2.4347e-01, -2.1078e-01,  1.6428e-01, -1.6066e-01, -1.1156e-01,
       -1.7952e-01,  7.9397e-01,  1.0435e-01, -4.8422e-01,  2.3403e-01,
        1.3215e-01, -3.4919e-02, -2.7259e-01,  9.3399e-02, -2.8221e-01,
       -2.9590e-01,  3.6580e-01,  1.1716e-01,  7.8769e-02,  2.3613e-01,
       -8.5098e-03, -1.2822e-01,  2.7307e-02, -3.6087e-01,  4.3706e-01,
        4.2840e-01, -2.8270e-02,  5.0164e-01, -2.0879e-02,  2.6959e-01,
       -2.2048e-01, -2.8754e-01, -2.2090e-01, -5.1641e-02,  1.3639e-01,
        5.4048e-01, -1.1889e-01,  3.0898e-01,  3.0809e-01, -2.2642e-01,
       -1.6974e-02,  7.9386e-02, -1.0483e-01,  8.7422e-02,  3.69

In [52]:
model_tweet_word['space'].shape

(200,)

In [53]:
np.dot(model_tweet_word['space'],model_tweet_word['pokemon']) #space in twitter language is more likely to refer to outer space

5.0909085

In [54]:
np.dot(model_complaint_to_tweet['space'],model_tweet_word['pokemon'])##space in complaint language is more likely to refer to area of a property

0.14141041

In [55]:
#https://stackoverflow.com/questions/18424228/cosine-similarity-between-2-number-lists
from numpy.linalg import norm

In [56]:
np.dot(model_tweet_word['u'],model_tweet_word['i'])/(norm(model_tweet_word['u'])*norm(model_tweet_word['i']))

0.6394821

In [57]:
np.dot(model_complaint_to_tweet['u'],model_tweet_word['i'])/(norm(model_complaint_to_tweet['u'])*norm(model_tweet_word['i']))

0.022840686

In [57]:
#http://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.cosine_similarity.html
#https://stackoverflow.com/questions/18424228/cosine-similarity-between-2-number-lists
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity(model_tweet_word['space'].reshape(-1,1), model_tweet_word['pokemon'].reshape(-1,1))

array([[-1.,  1., -1., ...,  1.,  1.,  1.],
       [-1.,  1., -1., ...,  1.,  1.,  1.],
       [-1.,  1., -1., ...,  1.,  1.,  1.],
       ...,
       [ 1., -1.,  1., ..., -1., -1., -1.],
       [ 1., -1.,  1., ..., -1., -1., -1.],
       [ 1., -1.,  1., ..., -1., -1., -1.]], dtype=float32)

In [58]:
model_complaint_to_tweet['blessed']-model_complaint_word['blessed']

array([ 0.514419  ,  0.0891369 ,  0.012198  ,  0.3288779 , -0.03441201,
       -0.22741789,  0.09411149,  0.15651   ,  0.15312201, -0.102647  ,
        0.0247923 , -0.1572145 , -0.277686  ,  0.1350866 , -0.3666261 ,
       -0.12745139, -0.456599  , -0.18104589, -0.025032  ,  0.122208  ,
       -0.019124  , -0.349333  , -0.4221945 , -0.3470322 ,  0.0027367 ,
       -0.52059203,  0.2628662 , -0.0623594 , -0.53476   ,  0.2616785 ,
       -0.16586918, -0.0435017 , -0.2713268 , -0.1760734 ,  0.5134021 ,
        0.25705847,  0.0898459 , -0.1362712 ,  0.22680001, -0.4895208 ,
       -0.22419049, -0.2144239 , -0.12611449,  0.2804015 ,  0.163714  ,
       -0.1408832 , -0.448236  , -0.37175   ,  0.4414677 , -0.3220468 ,
       -0.1011443 ,  0.00137397,  0.20548047, -0.2606531 ,  0.3395483 ,
       -0.48139668, -0.1585074 ,  0.3078371 , -0.1434567 ,  0.20546749,
       -0.3003196 ,  0.22754282,  0.1591917 ,  0.1845073 ,  0.3821815 ,
        0.3222886 , -0.28911307,  0.10569585, -0.2900448 , -0.17

In [59]:
model_complaint_to_tweet['success']-model_complaint_word['success']

array([-3.05792004e-01, -2.51249969e-02, -1.64709002e-01, -1.15318693e-01,
        3.06567788e-01, -1.41098008e-01, -3.24621022e-01,  1.40366405e-01,
        2.48251498e-01, -1.46695197e-01,  2.73413002e-01, -3.18869680e-01,
        1.94090977e-02, -2.46226490e-01,  2.00990707e-01, -2.91014984e-02,
        2.39022002e-02, -4.61980194e-01, -5.24080336e-01, -3.68287802e-01,
        4.79151815e-01, -5.91064990e-02,  1.32027403e-01,  8.50235000e-02,
       -1.20136298e-01,  1.52791291e-01,  9.15230066e-03,  1.48900002e-01,
        1.77332014e-02, -1.03553206e-01, -2.22814009e-01, -1.89529896e-01,
       -2.10397691e-01, -6.97430000e-02, -2.66330510e-01, -4.29574013e-01,
        3.64917010e-01, -3.53945613e-01,  1.42998010e-01, -3.59090120e-01,
        1.15920201e-01,  7.21504539e-02, -2.74753809e-01,  3.31487298e-01,
        3.32397193e-01, -3.14476877e-01, -4.43817377e-01, -4.16334808e-01,
        3.08887035e-01, -4.36991006e-01,  4.75464985e-02,  2.23338291e-01,
        4.56743300e-01,  

In [58]:
np.dot(model_complaint_to_tweet['success'],model_complaint_word['success'])/(norm(model_complaint_to_tweet['success'])*norm(model_complaint_word['success']))

-0.021623872

In [59]:
np.dot(model_complaint_to_tweet['blessed'],model_complaint_word['blessed'])/(norm(model_complaint_to_tweet['blessed'])*norm(model_complaint_word['blessed']))

-0.029421806

In [60]:
np.dot(model_complaint_word['success'],model_complaint_word['blessed'])/(norm(model_complaint_word['success'])*norm(model_complaint_word['blessed']))

0.28567168

In [61]:
np.dot(model_complaint_to_tweet['success'],model_complaint_to_tweet['blessed'])/(norm(model_complaint_to_tweet['success'])*norm(model_complaint_to_tweet['blessed']))

0.22313003

In [62]:
model_complaint_to_tweet.similarity('success','blessed')

0.22313006

In [63]:
np.dot(model_complaint_to_tweet['blessed'],model_tweet_word['pokemon'])/(norm(model_complaint_to_tweet['blessed'])*norm(model_tweet_word['pokemon']))

-0.017767342

In [64]:
def vec_in_cross_domain(word):
    try:
        return model_complaint_to_tweet[word]
    except:
        try:
            return model_tweet_word[word]
        except:
            return np.NaN
        


In [65]:
def similarity_in_cross_domain(word_1,word_2):
    #checking for the case where a word vector in an array of np.NaN values or the word is a np.NaN value
    if pd.isnull(pd.Series(word_1)).all():
        return np.NaN
    if pd.isnull(pd.Series(word_2)).all():
        return np.NaN
    
    if type(word_1)==np.ndarray:
        word_vector_1=word_1
    else:
        word_vector_1=vec_in_cross_domain(word_1)

    if pd.isnull(pd.Series(word_vector_1)).all():
        return np.NaN
    
    if type(word_2)==np.ndarray:
        word_vector_2=word_2
    else:
        word_vector_2=vec_in_cross_domain(word_2)

    if pd.isnull(pd.Series(word_vector_2)).all():
        return np.NaN        
    
    return(np.dot(word_vector_1,word_vector_2)/(norm(word_vector_1)*norm(word_vector_2)))

In [ ]:
#ignore this cell
def similarity_in_cross_domain(word_1,word_2):
    #checking for the case where a word vector in an array of np.NaN values or the word is a np.NaN value
    if pd.isnull(pd.Series(word_1)).all():
        return np.NaN
    if pd.isnull(pd.Series(word_2)).all():
        return np.NaN
    
    if type(word_1)==np.ndarray:
        word_vector_1=word_1
    else:
        word_vector_1=vec_in_cross_domain(word_1)

    if type(word_2)==np.ndarray:
        word_vector_2=word_2
    else:
        word_vector_2=vec_in_cross_domain(word_2)
    
    try:
        return(np.dot(word_vector_1,word_vector_2)/(norm(word_vector_1)*norm(word_vector_2)))
    except:
        return np.NaN

In [66]:
similarity_in_cross_domain('blessed','pokemon')

-0.017767342

In [67]:
df_more_agreeable=pd.read_csv('./data/world_wellbeing_proc/MoreAgreeable.csv')

In [68]:
df_more_agreeable.shape

(100, 2)

In [69]:
df_more_agreeable.head()

,Word,Correlation
0,birthday wishes !,0.036052
1,at church,0.036221
2,joy,0.036244
3,grateful,0.036311
4,hope everyone,0.036494


In [70]:
df_more_agreeable['Word Vector']=df_more_agreeable['Word'].apply(lambda x:vec_in_cross_domain(x))

In [71]:
df_more_agreeable.head()

,Word,Correlation,Word Vector
0,birthday wishes !,0.036052,NaN
1,at church,0.036221,NaN
2,joy,0.036244,"[0.141265, -0.118765, -0.266715, -0.101807, -0..."
3,grateful,0.036311,"[0.153888, 0.0477556, -0.179458, -0.131209, 0...."
4,hope everyone,0.036494,NaN


In [72]:
df_more_agreeable.dropna(inplace=True)

In [73]:
df_more_agreeable.head()

,Word,Correlation,Word Vector
2,joy,0.036244,"[0.141265, -0.118765, -0.266715, -0.101807, -0..."
3,grateful,0.036311,"[0.153888, 0.0477556, -0.179458, -0.131209, 0...."
8,happy,0.036828,"[0.128521, -0.0364565, -0.26435, -0.107667, 0...."
13,isaiah,0.036972,"[0.090046, 0.23689, 0.52641, 0.1598, 0.31475, ..."
15,faith,0.037415,"[0.12209, 0.129739, -0.0500334, -0.123272, 0.0..."


In [74]:
df_more_agreeable.shape

(35, 3)

In [75]:
more_agreeable_corr_sum=df_more_agreeable['Correlation'].sum()

In [76]:
more_agreeable_corr_sum

1.530637

In [77]:
df_more_agreeable['Word Vector adj by correlation']=df_more_agreeable['Word Vector']*df_more_agreeable['Correlation']/more_agreeable_corr_sum

In [78]:
df_more_agreeable.head()

,Word,Correlation,Word Vector,Word Vector adj by correlation
2,joy,0.036244,"[0.141265, -0.118765, -0.266715, -0.101807, -0...","[0.0033450185, -0.0028122398, -0.006315552, -0..."
3,grateful,0.036311,"[0.153888, 0.0477556, -0.179458, -0.131209, 0....","[0.0036506548, 0.0011328966, -0.004257247, -0...."
8,happy,0.036828,"[0.128521, -0.0364565, -0.26435, -0.107667, 0....","[0.0030922885, -0.0008771642, -0.0063604116, -..."
13,isaiah,0.036972,"[0.090046, 0.23689, 0.52641, 0.1598, 0.31475, ...","[0.0021750296, 0.0057219947, 0.012715249, 0.00..."
15,faith,0.037415,"[0.12209, 0.129739, -0.0500334, -0.123272, 0.0...","[0.0029843766, 0.0031713496, -0.00122302, -0.0..."


In [79]:
df_more_agreeable['Word Vector adj by correlation'].sum()

array([ 0.03732684, -0.00437816, -0.1145325 ,  0.02668251,  0.13286865,
        0.00664927, -0.03402846,  0.05776181, -0.06943931,  0.02091441,
       -0.14408197,  0.04916506, -0.10219061, -0.00686401,  0.08924537,
        0.15890972, -0.10310023, -0.05916058,  0.05467194,  0.07579491,
       -0.05955235,  0.01207369,  0.02288588, -0.12566797,  0.00926388,
        0.01876919,  0.08051121,  0.01009621, -0.03363694,  0.02789659,
        0.13781454,  0.02287706, -0.14524771,  0.03875488, -0.04800779,
        0.13658132,  0.18888417,  0.04852261, -0.03838851, -0.07661507,
        0.11023237,  0.12729247,  0.0052919 ,  0.05438076, -0.04215462,
        0.08301628, -0.16575421, -0.09322493, -0.04862462, -0.09155285,
        0.00364633,  0.02919299, -0.09047845,  0.09164458,  0.03839822,
       -0.01806715,  0.0147088 ,  0.0347864 , -0.03689391, -0.06474417,
        0.0735404 , -0.00825975, -0.03769395,  0.11017784,  0.08286597,
        0.02929308, -0.02201286, -0.02421247,  0.069716  ,  0.14

In [80]:
df_more_agreeable['Word Vector adj by correlation'].sum().shape

(200,)

In [81]:
def representative_trait_vec(filepath):
    df_trait=pd.read_csv(filepath)
    df_trait['Word Vector']=df_trait['Word'].apply(lambda x:vec_in_cross_domain(x))
    df_trait.dropna(inplace=True)
    trait_corr_sum=df_trait['Correlation'].sum()
    df_trait['Word Vector adj by correlation']=df_trait['Word Vector']*df_trait['Correlation']/trait_corr_sum
    return df_trait['Word Vector adj by correlation'].sum()
    

In [82]:
representative_trait_vec('./data/world_wellbeing_proc/MoreAgreeable.csv')

array([ 0.03732684, -0.00437816, -0.1145325 ,  0.02668251,  0.13286865,
        0.00664927, -0.03402846,  0.05776181, -0.06943931,  0.02091441,
       -0.14408197,  0.04916506, -0.10219061, -0.00686401,  0.08924537,
        0.15890972, -0.10310023, -0.05916058,  0.05467194,  0.07579491,
       -0.05955235,  0.01207369,  0.02288588, -0.12566797,  0.00926388,
        0.01876919,  0.08051121,  0.01009621, -0.03363694,  0.02789659,
        0.13781454,  0.02287706, -0.14524771,  0.03875488, -0.04800779,
        0.13658132,  0.18888417,  0.04852261, -0.03838851, -0.07661507,
        0.11023237,  0.12729247,  0.0052919 ,  0.05438076, -0.04215462,
        0.08301628, -0.16575421, -0.09322493, -0.04862462, -0.09155285,
        0.00364633,  0.02919299, -0.09047845,  0.09164458,  0.03839822,
       -0.01806715,  0.0147088 ,  0.0347864 , -0.03689391, -0.06474417,
        0.0735404 , -0.00825975, -0.03769395,  0.11017784,  0.08286597,
        0.02929308, -0.02201286, -0.02421247,  0.069716  ,  0.14

In [83]:
more_agreeable_rep_vec=representative_trait_vec('./data/world_wellbeing_proc/MoreAgreeable.csv')
less_agreeable_rep_vec=representative_trait_vec('./data/world_wellbeing_proc/LessAgreeable.csv')
more_conscientious_rep_vec=representative_trait_vec('./data/world_wellbeing_proc/MoreConscientious.csv')
less_conscientious_rep_vec=representative_trait_vec('./data/world_wellbeing_proc/LessConscientious.csv')
more_open_rep_vec=representative_trait_vec('./data/world_wellbeing_proc/MoreOpen.csv')
less_open_rep_vec=representative_trait_vec('./data/world_wellbeing_proc/LessOpen.csv')
more_extraversion_rep_vec=representative_trait_vec('./data/world_wellbeing_proc/MoreExtraversion.csv')
less_extraversion_rep_vec=representative_trait_vec('./data/world_wellbeing_proc/LessExtraversion.csv')
more_neurotic_rep_vec=representative_trait_vec('./data/world_wellbeing_proc/MoreNeurotic.csv')
less_neurotic_rep_vec=representative_trait_vec('./data/world_wellbeing_proc/LessNeurotic.csv')

In [84]:
less_neurotic_rep_vec

array([ 1.22389212e-01, -6.44454956e-02, -2.06510991e-01, -1.09609857e-01,
       -1.08566489e-02,  1.09011307e-04,  5.00229858e-02,  5.42263091e-02,
       -6.41923398e-02,  8.35816469e-03,  1.40416268e-02,  9.14378688e-02,
       -7.04651847e-02,  3.40258479e-02, -1.95799936e-02, -6.82388619e-03,
       -2.11720057e-02, -7.13331476e-02,  2.35917121e-02, -1.50473407e-02,
       -3.64125110e-02,  4.43119975e-03,  3.44286151e-02, -4.09001578e-03,
       -1.88354366e-02,  6.71250820e-02,  2.92269979e-02,  6.04801886e-02,
       -6.70294687e-02,  2.60401610e-02,  5.32942936e-02,  2.93314662e-02,
       -1.48835657e-02, -8.37900024e-03, -7.03096986e-02,  3.52899916e-02,
        8.09915885e-02, -3.35049555e-02, -3.08652110e-02,  9.30484571e-03,
        1.28614316e-02,  3.81267741e-02, -4.91251796e-02,  1.47645203e-02,
       -1.93767212e-02,  2.58478839e-02, -7.90760815e-02, -2.73448974e-02,
       -6.76651448e-02,  3.37940343e-02,  1.92089956e-02, -5.09752594e-02,
       -6.33538887e-02,  

In [85]:
type(less_neurotic_rep_vec)

numpy.ndarray

# Neuroticism check:

In [86]:
similarity_in_cross_domain('success',less_neurotic_rep_vec)

0.078945674

In [87]:
similarity_in_cross_domain('success',more_neurotic_rep_vec)

0.14464492

In [88]:
similarity_in_cross_domain('mountains',more_neurotic_rep_vec),similarity_in_cross_domain('mountains',less_neurotic_rep_vec)

(-0.09435735, 0.15275943)

In [89]:
similarity_in_cross_domain('hurts',more_neurotic_rep_vec),similarity_in_cross_domain('hurts',less_neurotic_rep_vec)

(-0.07830979, 0.29812363)

In [90]:
similarity_in_cross_domain('pills',more_neurotic_rep_vec),similarity_in_cross_domain('pills',less_neurotic_rep_vec)

(-0.07218527, 0.35100695)

In [91]:
similarity_in_cross_domain('depressed',more_neurotic_rep_vec),similarity_in_cross_domain('depressed',less_neurotic_rep_vec)

(-0.088119656, 0.37950018)

# Extraversion Check:

In [92]:
similarity_in_cross_domain('party',more_extraversion_rep_vec),similarity_in_cross_domain('party',less_extraversion_rep_vec)

(-0.12212039, -0.07771775)

In [93]:
similarity_in_cross_domain('gym',more_neurotic_rep_vec),similarity_in_cross_domain('gym',less_neurotic_rep_vec)

(-0.013864824, 0.116652265)

In [94]:
similarity_in_cross_domain("didn't",more_neurotic_rep_vec),similarity_in_cross_domain("didn't",less_neurotic_rep_vec)

(-0.07631136, 0.121811554)

# Agreeableness Check:

In [95]:
similarity_in_cross_domain('excited',more_agreeable_rep_vec),similarity_in_cross_domain('excited',less_agreeable_rep_vec)

(0.1967261, 0.12211669)

In [96]:
similarity_in_cross_domain('joy',more_agreeable_rep_vec),similarity_in_cross_domain('joy',less_agreeable_rep_vec)

(0.19337471, 0.06937123)

In [97]:
similarity_in_cross_domain('damned',more_agreeable_rep_vec),similarity_in_cross_domain('damned',less_agreeable_rep_vec)

(0.0553869, -0.009491111)

In [98]:
#this is for sanity check how well the representative trait matrix works for word in the World Well
def representative_trait_vec_based_est_scores(filepath,representative_trait_vec):
    df_trait_est_score=pd.read_csv(filepath)
    df_trait_est_score['Estimated Score']=df_trait_est_score['Word'].apply(lambda x:similarity_in_cross_domain(x,representative_trait_vec))
    df_trait_est_score.dropna(inplace=True)
    return df_trait_est_score

In [99]:
#this is for sanity check how well the representative trait matrix works for word in the World WellBeing Project
more_agreeable_est_scores=representative_trait_vec_based_est_scores('./data/world_wellbeing_proc/MoreAgreeable.csv',more_agreeable_rep_vec)
less_agreeable_est_scores=representative_trait_vec_based_est_scores('./data/world_wellbeing_proc/LessAgreeable.csv',less_agreeable_rep_vec)
more_conscientious_est_scores=representative_trait_vec_based_est_scores('./data/world_wellbeing_proc/MoreConscientious.csv',more_conscientious_rep_vec)
less_conscientious_est_scores=representative_trait_vec_based_est_scores('./data/world_wellbeing_proc/LessConscientious.csv',less_conscientious_rep_vec)
more_open_est_scores=representative_trait_vec_based_est_scores('./data/world_wellbeing_proc/MoreOpen.csv',more_open_rep_vec)
less_open_est_scores=representative_trait_vec_based_est_scores('./data/world_wellbeing_proc/LessOpen.csv',less_open_rep_vec)
more_extraversion_est_scores=representative_trait_vec_based_est_scores('./data/world_wellbeing_proc/MoreExtraversion.csv',more_extraversion_rep_vec)
less_extraversion_est_scores=representative_trait_vec_based_est_scores('./data/world_wellbeing_proc/LessExtraversion.csv',less_extraversion_rep_vec)
more_neurotic_est_scores=representative_trait_vec_based_est_scores('./data/world_wellbeing_proc/MoreNeurotic.csv',more_neurotic_rep_vec)
less_neurotic_est_scores=representative_trait_vec_based_est_scores('./data/world_wellbeing_proc/LessNeurotic.csv',less_neurotic_rep_vec)

In [100]:
more_agreeable_est_scores

,Word,Correlation,Estimated Score
2,joy,0.036244,0.193375
3,grateful,0.036311,0.283878
8,happy,0.036828,0.256743
13,isaiah,0.036972,0.630684
15,faith,0.037415,0.036553
16,thank,0.037443,0.283264
17,loving,0.037457,0.112384
19,family,0.037646,0.147604
21,spring,0.038180,0.045008
23,tomorrow,0.038442,0.037359


# It seems that the accuracy of the personality trait classification may not be good...but even if it is 60-70% it could still be potentially useful...and if not useful...it will get ignored in the matrix factorization (as it would look at generalizations...)

In [101]:
dfmortproddebtcoll_common_cos_ratings
dfstudproddebtcoll_common_cos_ratings
dfpayproddebtcoll_common_cos_ratings

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,...,141,142,143,144,145,146,147,148,149,150
0,2016-12-03,Payday loan,NaN,Charged bank acct wrong day or amt,NaN,Collections agency was giving the wrong inform...,NaN,Ad Astra Recovery Services Inc,LA,707XX,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,-1.875
1,2015-09-02,Payday loan,NaN,Charged fees or interest I didn't expect,NaN,They Keep HARASSING me using fake numbers from...,NaN,ADR ARBITRATION SERVICES (Closed),TX,760XX,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,-1.750
3,2016-01-31,Payday loan,NaN,Charged fees or interest I didn't expect,NaN,I TOOK OUT A PAYDAY LOAN WITH ADVANCE FINANCIA...,Company believes it acted appropriately as aut...,"Harpeth Financial Services, LLC",TN,370XX,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.257728,0.0,-2.875
4,2016-04-27,Payday loan,NaN,Can't stop charges to bank account,NaN,"Evergreen Financial Investments , Inc DBA Rapi...",NaN,CURO Intermediate Holdings,WA,986XX,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,-0.125
5,2016-02-25,Payday loan,NaN,Received a loan I didn't apply for,NaN,They saying I had got a pay day loan but I did...,Company believes complaint caused principally ...,"Enova International, Inc.",AL,352XX,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000
6,2015-08-26,Payday loan,NaN,Charged fees or interest I didn't expect,NaN,"The loan was for XXXX, I was charged XXXX mont...",Company believes it acted appropriately as aut...,"LDF Holdings, LLC",PA,179XX,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000
7,2016-02-23,Payday loan,NaN,Received a loan I didn't apply for,NaN,"On Sunday XXXX XXXX, 2016 I was called by Cash...",NaN,"COMMUNITY CHOICE FINANCIAL, INC.",TX,752XX,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,1.250
8,2016-09-27,Payday loan,NaN,Charged fees or interest I didn't expect,NaN,hi. i have loaned money from ace cash express ...,NaN,"ACE CASH EXPRESS, INC.",MN,551XX,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.625
9,2017-01-04,Payday loan,NaN,Received a loan I didn't apply for,NaN,My identity was stolen in XX/XX/2009. A XXXX X...,NaN,"Mobiloans, LLC",FL,331XX,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.625
10,2015-04-02,Payday loan,NaN,Charged fees or interest I didn't expect,NaN,I was in a bind and took out a loan with Castl...,NaN,"Big Picture Loans, LLC",OK,746XX,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,1.125


In [102]:
dfmortproddebtcoll_common_cos_ratings

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,...,141,142,143,144,145,146,147,148,149,150
1,2015-07-16,Mortgage,Conventional fixed mortgage,"Application, originator, mortgage broker",NaN,"In XXXX, I and my ex-husband applied for a ref...",NaN,HSBC NORTH AMERICA HOLDINGS INC.,IL,625XX,...,0.000000,0.000000,0.0,0.0,0.00000,0.0,0.0,0.000000,0.0,1.875
3,2015-03-24,Mortgage,Conventional fixed mortgage,Settlement process and costs,NaN,I got recent modification ( XXXX/XXXX/2015 ) f...,NaN,OCWEN LOAN SERVICING LLC,TX,774XX,...,0.000000,0.000000,0.0,0.0,0.00000,0.0,0.0,0.000000,0.0,2.000
4,2015-03-30,Mortgage,Conventional fixed mortgage,Settlement process and costs,NaN,I filed a complain against Chase Mortgage on X...,NaN,JPMORGAN CHASE & CO.,MD,207XX,...,0.000000,0.000000,0.0,0.0,0.00000,0.0,0.0,0.000000,0.0,0.625
5,2015-07-06,Mortgage,Conventional fixed mortgage,Settlement process and costs,NaN,XXXX previous case numberSince my last written...,NaN,NATIONSTAR MORTGAGE,GA,305XX,...,0.000000,0.000000,0.0,0.0,0.00000,0.0,0.0,0.000000,0.0,0.125
6,2015-09-29,Mortgage,Conventional fixed mortgage,"Application, originator, mortgage broker",NaN,I received exceptional service from my lender ...,Company chooses not to provide a public response,"SUNTRUST BANKS, INC.",DC,200XX,...,0.000000,0.000000,0.0,0.0,0.00000,0.0,0.0,0.000000,0.0,0.500
7,2016-12-14,Mortgage,Conventional fixed mortgage,"Loan modification,collection,foreclosure",NaN,This loan closed as a short sale with XXXX XXX...,NaN,Selene Finance LP,WA,984XX,...,0.000000,0.000000,0.0,0.0,0.00000,0.0,0.0,0.000000,0.0,0.625
8,2015-04-28,Mortgage,Conventional fixed mortgage,"Loan servicing, payments, escrow account",NaN,( To preface this may be nothing but it is fro...,NaN,"Seterus, Inc.",MP,486XX,...,0.000000,0.000000,0.0,0.0,0.00000,0.0,0.0,0.000000,0.0,1.375
9,2016-01-22,Mortgage,Conventional fixed mortgage,"Loan modification,collection,foreclosure",NaN,In XXXX XXXX my wife and I applied through the...,Company believes it acted appropriately as aut...,"SELECT PORTFOLIO SERVICING, INC.",NaN,NaN,...,0.000000,0.000000,0.0,0.0,0.00000,0.0,0.0,0.000000,0.0,0.625
10,2017-01-24,Mortgage,Conventional fixed mortgage,"Loan modification,collection,foreclosure",NaN,Submitted a short sale package to Loan Care vi...,Company has responded to the consumer and the ...,"LoanCare, LLC",FL,330XX,...,0.000000,0.000000,0.0,0.0,0.00000,0.0,0.0,0.000000,0.0,4.500
11,2016-04-04,Mortgage,Conventional fixed mortgage,"Loan servicing, payments, escrow account",NaN,the day after I closed on a new loan it was so...,Company believes complaint is the result of an...,"Shellpoint Partners, LLC",TX,786XX,...,0.000000,0.000000,0.0,0.0,0.00000,0.0,0.0,0.000000,0.0,0.125


In [103]:
def complaint_trait_est_score(x,trait_rep_vec):
    sum=0
    word_count=1 #intialized as 1 in order to avoid division be zero problem
    for word in nlp(x):
        if not(pd.isnull(pd.Series(vec_in_cross_domain(word.text))).all()): #This is required so that word not part of the cross domain word vector space does not get considered as np.NaN which then means that for such a single word in the complaint text the entire score become an array of np.NaN values (resulting from the similarity computation) 
            word_trait_est_score=similarity_in_cross_domain(word.text,trait_rep_vec)
            sum=sum+word_trait_est_score
            word_count=word_count+1
    return sum/word_count
        
    

In [104]:
pd.DataFrame(more_agreeable_rep_vec).to_pickle('./data/more_agreeable_rep_vec.pickle')
pd.DataFrame(less_agreeable_rep_vec).to_pickle('./data/less_agreeable_rep_vec.pickle')
pd.DataFrame(more_conscientious_rep_vec).to_pickle('./data/more_conscientious_rep_vec.pickle')
pd.DataFrame(less_conscientious_rep_vec).to_pickle('./data/less_conscientious_rep_vec.pickle')
pd.DataFrame(more_open_rep_vec).to_pickle('./data/more_open_rep_vec.pickle')
pd.DataFrame(less_open_rep_vec).to_pickle('./data/less_open_rep_vec.pickle')
pd.DataFrame(more_extraversion_rep_vec).to_pickle('./data/more_extraversion_rep_vec.pickle')
pd.DataFrame(less_extraversion_rep_vec).to_pickle('./data/less_extraversion_rep_vec.pickle')
pd.DataFrame(more_neurotic_rep_vec).to_pickle('./data/more_neurotic_rep_vec.pickle')
pd.DataFrame(less_neurotic_rep_vec).to_pickle('./data/less_neurotic_rep_vec.pickle')

# Note the numpy arrays have been converted into DataFrames and then pickled.

# Running Python script instead of this Notebook...(21-Oct-18: 1542hrs IST)
Python script run on my Windows laptop, Burrow and BigRed2. The only difference in the run on this laptop that an older version of spaCy was used.

# See 6B.py

# Got output from laptop on 22-Oct-18 morning; also got output on burrow; and job got executed faster on BigRed2 after it began running later the same week.

In [3]:
dfmortproddebtcoll_common_cos_ratings_traits=pd.read_pickle('./data/dfmortproddebtcoll_common_cos_ratings_traits.pickle')
dfpayproddebtcoll_common_cos_ratings_traits=pd.read_pickle('./data/dfpayproddebtcoll_common_cos_ratings_traits.pickle')
dfstudproddebtcoll_common_cos_ratings_traits=pd.read_pickle('./data/dfstudproddebtcoll_common_cos_ratings_traits.pickle')

In [5]:
dfmortproddebtcoll_common_cos_ratings_traits

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,...,Less Extraversion,More Extraversion,More Neuroticism,Less Neuroticism,More Agreeableness,Less Agreeableness,More Conscientiousness,Less Conscientiousness,More Openness,Less Openness
1,2015-07-16,Mortgage,Conventional fixed mortgage,"Application, originator, mortgage broker",NaN,"In XXXX, I and my ex-husband applied for a ref...",NaN,HSBC NORTH AMERICA HOLDINGS INC.,IL,625XX,...,-0.021339,0.058904,0.025495,-0.003002,0.008638,-0.002557,0.056898,-0.019448,0.083322,-0.008102
3,2015-03-24,Mortgage,Conventional fixed mortgage,Settlement process and costs,NaN,I got recent modification ( XXXX/XXXX/2015 ) f...,NaN,OCWEN LOAN SERVICING LLC,TX,774XX,...,-0.034974,0.070715,0.034426,-0.021819,0.025378,0.006242,0.047532,-0.008081,0.028724,-0.009535
4,2015-03-30,Mortgage,Conventional fixed mortgage,Settlement process and costs,NaN,I filed a complain against Chase Mortgage on X...,NaN,JPMORGAN CHASE & CO.,MD,207XX,...,-0.005036,0.093421,0.019455,0.016975,0.012096,-0.009140,0.074243,-0.006334,0.124311,0.012286
5,2015-07-06,Mortgage,Conventional fixed mortgage,Settlement process and costs,NaN,XXXX previous case numberSince my last written...,NaN,NATIONSTAR MORTGAGE,GA,305XX,...,-0.006813,0.065042,0.020585,-0.000491,0.001900,-0.013571,0.063852,-0.019208,0.104267,0.005903
6,2015-09-29,Mortgage,Conventional fixed mortgage,"Application, originator, mortgage broker",NaN,I received exceptional service from my lender ...,Company chooses not to provide a public response,"SUNTRUST BANKS, INC.",DC,200XX,...,-0.010780,0.089203,-0.004685,0.056045,0.004103,-0.008782,0.091999,-0.008167,0.185642,-0.013309
7,2016-12-14,Mortgage,Conventional fixed mortgage,"Loan modification,collection,foreclosure",NaN,This loan closed as a short sale with XXXX XXX...,NaN,Selene Finance LP,WA,984XX,...,-0.040883,0.049446,0.028321,-0.054641,-0.015928,-0.015860,0.026428,-0.033976,0.016433,-0.012889
8,2015-04-28,Mortgage,Conventional fixed mortgage,"Loan servicing, payments, escrow account",NaN,( To preface this may be nothing but it is fro...,NaN,"Seterus, Inc.",MP,486XX,...,-0.025734,0.067790,0.028420,-0.032020,0.004896,0.000533,0.042900,-0.009838,0.033672,-0.000111
9,2016-01-22,Mortgage,Conventional fixed mortgage,"Loan modification,collection,foreclosure",NaN,In XXXX XXXX my wife and I applied through the...,Company believes it acted appropriately as aut...,"SELECT PORTFOLIO SERVICING, INC.",NaN,NaN,...,-0.021068,0.049210,0.029097,-0.008554,0.012540,0.007017,0.064406,-0.017712,0.060580,-0.008735
10,2017-01-24,Mortgage,Conventional fixed mortgage,"Loan modification,collection,foreclosure",NaN,Submitted a short sale package to Loan Care vi...,Company has responded to the consumer and the ...,"LoanCare, LLC",FL,330XX,...,0.001404,0.084649,0.026884,-0.000540,0.005003,-0.002547,0.075713,-0.004791,0.092318,0.019572
11,2016-04-04,Mortgage,Conventional fixed mortgage,"Loan servicing, payments, escrow account",NaN,the day after I closed on a new loan it was so...,Company believes complaint is the result of an...,"Shellpoint Partners, LLC",TX,786XX,...,-0.001170,0.114161,0.022214,0.050578,0.011356,-0.002631,0.089315,-0.000647,0.168887,0.013651


In [6]:
dfpayproddebtcoll_common_cos_ratings_traits

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,...,More Extraversion,Less Extraversion,More Neuroticism,Less Neuroticism,More Agreeableness,Less Agreeableness,More Conscientiousness,Less Conscientiousness,More Openness,Less Openness
0,2016-12-03,Payday loan,NaN,Charged bank acct wrong day or amt,NaN,Collections agency was giving the wrong inform...,NaN,Ad Astra Recovery Services Inc,LA,707XX,...,0.047698,-0.006502,0.015057,-0.042300,-0.014332,-0.011392,0.027784,-0.011628,0.054910,0.000507
1,2015-09-02,Payday loan,NaN,Charged fees or interest I didn't expect,NaN,They Keep HARASSING me using fake numbers from...,NaN,ADR ARBITRATION SERVICES (Closed),TX,760XX,...,0.102972,0.015709,0.032437,0.042906,0.016593,0.002773,0.083651,0.011804,0.142531,0.020235
3,2016-01-31,Payday loan,NaN,Charged fees or interest I didn't expect,NaN,I TOOK OUT A PAYDAY LOAN WITH ADVANCE FINANCIA...,Company believes it acted appropriately as aut...,"Harpeth Financial Services, LLC",TN,370XX,...,-0.043123,-0.068670,-0.021912,-0.120010,-0.002100,-0.066773,-0.086484,-0.049323,-0.242168,0.003428
4,2016-04-27,Payday loan,NaN,Can't stop charges to bank account,NaN,"Evergreen Financial Investments , Inc DBA Rapi...",NaN,CURO Intermediate Holdings,WA,986XX,...,0.087809,-0.005495,0.024317,0.009081,0.003538,0.004802,0.064486,0.002337,0.096855,0.008070
5,2016-02-25,Payday loan,NaN,Received a loan I didn't apply for,NaN,They saying I had got a pay day loan but I did...,Company believes complaint caused principally ...,"Enova International, Inc.",AL,352XX,...,0.155233,-0.008475,0.017312,0.080614,0.015414,-0.006652,0.136308,0.000835,0.187985,0.015645
6,2015-08-26,Payday loan,NaN,Charged fees or interest I didn't expect,NaN,"The loan was for XXXX, I was charged XXXX mont...",Company believes it acted appropriately as aut...,"LDF Holdings, LLC",PA,179XX,...,0.125064,-0.005409,0.009401,0.053724,0.001115,-0.009954,0.091700,-0.006875,0.159096,0.018828
7,2016-02-23,Payday loan,NaN,Received a loan I didn't apply for,NaN,"On Sunday XXXX XXXX, 2016 I was called by Cash...",NaN,"COMMUNITY CHOICE FINANCIAL, INC.",TX,752XX,...,0.076050,-0.029405,0.024435,-0.029864,0.002249,-0.014303,0.054539,-0.020212,0.053398,-0.005895
8,2016-09-27,Payday loan,NaN,Charged fees or interest I didn't expect,NaN,hi. i have loaned money from ace cash express ...,NaN,"ACE CASH EXPRESS, INC.",MN,551XX,...,0.086057,-0.014598,0.017901,0.045945,0.003857,0.015255,0.069687,0.002138,0.122856,-0.007077
9,2017-01-04,Payday loan,NaN,Received a loan I didn't apply for,NaN,My identity was stolen in XX/XX/2009. A XXXX X...,NaN,"Mobiloans, LLC",FL,331XX,...,0.041714,-0.025809,0.009750,-0.045420,-0.008237,-0.019688,0.024012,-0.027255,0.018514,0.007054
10,2015-04-02,Payday loan,NaN,Charged fees or interest I didn't expect,NaN,I was in a bind and took out a loan with Castl...,NaN,"Big Picture Loans, LLC",OK,746XX,...,0.075632,-0.010694,0.011972,0.006143,-0.008742,-0.003783,0.038676,-0.004653,0.074688,0.005173


In [7]:
dfstudproddebtcoll_common_cos_ratings_traits

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,...,More Extraversion,Less Extraversion,More Neuroticism,Less Neuroticism,More Agreeableness,Less Agreeableness,More Conscientiousness,Less Conscientiousness,More Openness,Less Openness
0,2017-02-06,Student loan,Federal student loan servicing,Dealing with my lender or servicer,Don't agree with fees charged,I took out a student loan back in XXXX in the ...,Company believes it acted appropriately as aut...,Texas Guaranteed,TX,750XX,...,0.083101,-0.025356,0.018916,-0.008139,-0.012072,-0.004746,0.049642,-0.015565,0.065821,0.003404
1,2016-12-07,Student loan,Federal student loan servicing,Dealing with my lender or servicer,Received bad information about my loan,My lender is Nelnet for Federal student loans....,NaN,"NELNET, INC.",NC,282XX,...,0.082845,-0.003330,0.024573,0.029478,-0.010053,0.013372,0.061123,0.007226,0.126168,0.003620
2,2016-09-21,Student loan,Federal student loan servicing,Dealing with my lender or servicer,Don't agree with fees charged,I have had a federal student loan since I star...,NaN,AES/PHEAA,MD,210XX,...,0.086279,-0.008499,0.031389,0.013693,0.001371,0.012088,0.070944,0.004623,0.095259,0.015999
3,2017-03-04,Student loan,Federal student loan servicing,Dealing with my lender or servicer,Received bad information about my loan,I believe I was wronged by Navient in several ...,NaN,"Navient Solutions, LLC.",CA,926XX,...,0.059504,-0.011182,0.011962,-0.016240,-0.017110,-0.004407,0.039812,-0.013534,0.078575,0.009433
4,2017-02-11,Student loan,Federal student loan servicing,Dealing with my lender or servicer,Need information about my balance/terms,I 'm dealing with Navient and it has been a ha...,NaN,"Navient Solutions, LLC.",AR,720XX,...,0.079362,-0.005846,0.007840,0.024148,-0.013938,-0.001376,0.053283,-0.002627,0.121724,0.004132
5,2017-01-20,Student loan,Federal student loan servicing,Dealing with my lender or servicer,Keep getting calls about my loan,I tried to make payments that I could afford b...,NaN,"Navient Solutions, LLC.",VA,221XX,...,0.099147,-0.010712,0.021671,0.039396,-0.004263,0.011684,0.077946,0.001619,0.117908,0.012220
6,2017-02-10,Student loan,Federal student loan servicing,Can't repay my loan,Can't decrease my monthly payments,The company does not work with me. I have been...,NaN,AES/PHEAA,TX,751XX,...,0.110766,-0.002009,0.032822,0.047156,0.016948,0.009521,0.097472,0.003339,0.126711,0.019858
7,2016-04-25,Student loan,Federal student loan servicing,Dealing with my lender or servicer,Received bad information about my loan,I borrowed over much time from XXXX ( my paren...,NaN,AES/PHEAA,NY,117XX,...,0.083576,-0.013390,0.023094,0.015949,0.012432,-0.001399,0.046582,-0.001237,0.063926,0.017878
8,2017-02-28,Student loan,Federal student loan servicing,Dealing with my lender or servicer,Keep getting calls about my loan,The loan provider took me out of deferment whi...,NaN,"Navient Solutions, LLC.",NC,282XX,...,0.057413,0.010514,0.006988,0.027560,-0.014410,-0.006647,0.080357,0.001542,0.120184,0.016500
9,2016-10-23,Student loan,Federal student loan servicing,Dealing with my lender or servicer,Having problems with customer service,I have a breach of contract borrowers defense ...,NaN,"Navient Solutions, LLC.",NV,891XX,...,0.041159,-0.010825,0.037514,0.008257,-0.004955,0.024484,0.052431,-0.005577,0.097773,0.008265
